# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [164]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [165]:
# Checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [166]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# Extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [167]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [168]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [169]:
# Create KEYSPACE "sparkify"
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)

except Exception as e:
    print(e)

#### Set Keyspace

In [170]:
# Set KEYSPACE "sparkify" as current
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [171]:
## Create table for Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## PRIMARY KEY (PK) selection explanations: \
## First of all, PK should be unique. SessionId and ItemInSession can uniquely identify the row in the table, because our query \
## requires information only from the session and sessionItem, moreover the sessionItem is unique inside the session. \
## Here we can choose how to partition our data: by sessionId only, or by sessionId and itemInSession. \
## Both, approaches are acceptable, but in case of partitioning by sessionId only you can get into situation when some sessions \
## is very active (with high amount of itemInSession) and all of them will landed in the single partition which gives your a hot spot. \
## In the other hand, partitioning by both sessionId and itemInSession gives you more uniform distribution and this is my choice.
createTableForQuery1 = """
    CREATE TABLE IF NOT EXISTS music_app_sessions(
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        song_length decimal,
        PRIMARY KEY ((sessionId, itemInSession)));
"""
try:
    session.execute(createTableForQuery1)
except Exception as e:
    print(e)
       

In [172]:
file = 'event_datafile_new.csv'

insertDataForQuery1 = """
    INSERT INTO music_app_sessions(sessionId, itemInSession, artist, song, song_length)
    VALUES(%s, %s, %s, %s, %s);
"""

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            # Choose required fields and convert them to proper datatypes for insertion
            session.execute(insertDataForQuery1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
except Exception as e:
    print(e)

#### Do a SELECT to verify that the data have been inserted into table for Query 1

In [173]:
## Verify query 1: Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
selectForQuery1 = """
    SELECT artist, song, song_length
    FROM music_app_sessions
    WHERE sessionId = 338
    AND itemInSession = 4;
"""
try:
    rows = session.execute(selectForQuery1)
except Exception as e:
    print(e)
    
column_labels = ('Artist', 'Song', 'Length')
result_df = pd.DataFrame(list(rows), columns=column_labels)
print(result_df)

      Artist                             Song    Length
0  Faithless  Music Matters (Mark Knight Dub)  495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH REMAINING QUERY
_Question_: Why should we repeat code for insertion and scan CSV file three times instead of single pass?

_Answer_: Because we can have errors on the insertion statements, for example, in the table 3 and this means, that all tables will have uncompleted data. This is bad idea, because tables shouldn't affect each other and business should be able to query against other two tables while you fix the broken insert statement on table 3.

In [174]:
## Create table for Query 1: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## PRIMARY KEY (PK) selection explanations: \
## First of all, PK should be unique. UserId, SessionId and ItemInSession can uniquely identify the row in the table, because our query \
## requires information about the current user during current session. \
## We can partition our data by UserId and SessionId which can give us acceptable distribution. Also we need clustering column itemInSession \
## which gurantees us uniqness of data inside session and allows us to order data as we needed.
createTableForQuery2 = """
    CREATE TABLE IF NOT EXISTS user_sessions(
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        user_first_name text,
        user_last_name text,
        PRIMARY KEY ((userId, sessionId), itemInSession))
        WITH CLUSTERING ORDER BY (itemInSession ASC);
"""
try:
    session.execute(createTableForQuery2)
except Exception as e:
    print(e)

In [175]:
file = 'event_datafile_new.csv'

insertDataForQuery2 = """
    INSERT INTO user_sessions(userId, sessionId, itemInSession, artist, song, user_first_name, user_last_name)
    VALUES(%s, %s, %s, %s, %s, %s, %s);
"""

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            # Choose required fields and convert them to proper datatypes for insertion
            session.execute(insertDataForQuery2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
except Exception as e:
    print(e)

In [176]:
## Verify Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
selectForQuery2 = """
    SELECT artist, song, user_first_name, user_last_name
    FROM user_sessions
    WHERE userid = 10
    AND sessionid = 182;
"""
try:
    rows = session.execute(selectForQuery2)
except Exception as e:
    print(e)
    
column_labels = ('Artist', 'Song', 'First name', 'Last name')
result_df = pd.DataFrame(list(rows), columns=column_labels)
print(result_df)           

              Artist                                               Song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  First name Last name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


In [177]:
## Create table for Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## PRIMARY KEY (PK) selection explanations: \
## There are only fields song, firstName and lastName in the table. In fact this columns does not guarantee the uniqness of the rows \
## in the entre dataset, but for current query we are intrested in the deduplication of data. For example, user "John Doe" may listen \
## song 'All Hands Against His Own' multiple times, but we want to output his name once. Apache Cassandra allows us to make this trick \
## just specify only song, firstName and lastName in the PK. \
## Table will be partitioned by the song. It is possible to add some other fields for better distribution but for the current query where we need \
## only unique usernames it is enough. FirstName and lastName will be used as clustering columns.
createTableForQuery3 = """
    CREATE TABLE IF NOT EXISTS song_listeners(
        song text,
        user_first_name text,
        user_last_name text,
        PRIMARY KEY (song, user_first_name, user_last_name));
"""
try:
    session.execute(createTableForQuery3)
except Exception as e:
    print(e)

In [178]:
file = 'event_datafile_new.csv'

insertDataForQuery3 = """
    INSERT INTO song_listeners(song, user_first_name, user_last_name)
    VALUES(%s, %s, %s);
"""
    
try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            # Choose required fields and convert them to proper datatypes for insertion
            session.execute(insertDataForQuery3, (line[9], line[1], line[4]))
except Exception as e:
    print(e)

In [179]:
## Verify Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
selectForQuery3 = """
    SELECT user_first_name, user_last_name
    FROM song_listeners
    WHERE song = 'All Hands Against His Own';
"""
try:
    rows = session.execute(selectForQuery3)
except Exception as e:
    print(e)

column_labels = ('First name', 'Last name')
result_df = pd.DataFrame(list(rows), columns=column_labels)
print(result_df)          

   First name Last name
0  Jacqueline     Lynch
1        Sara   Johnson
2       Tegan    Levine


### Drop the tables before closing out the sessions

In [180]:
## TO-DO: Drop the table before closing out the sessions
dropTableForQuery1 = "DROP TABLE IF EXISTS music_app_sessions;"
dropTableForQuery2 = "DROP TABLE IF EXISTS user_sessions;"
dropTableForQuery3 = "DROP TABLE IF EXISTS song_listeners;"

try:
    session.execute(dropTableForQuery1)
    session.execute(dropTableForQuery2)
    session.execute(dropTableForQuery3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [181]:
session.shutdown()
cluster.shutdown()